# **Pre Entrega**
## **Alumna: Romina Galván**


---


---
# ETAPA 1
---

## 1) Crear un Documento en **Google Colaboratory** y cargar los sets de datos como DataFrames



In [ ]:
# Importando paquetes necesarios que luego seran llamados con nombres reducidos
import pandas as pd
import numpy as np
from IPython.display import display, HTML
# Montando la carpeta de Drive en Google Colab
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
# Listando el contenido de cierta ruta y chequeando que estan los dataset necesarios para el trabajo
# Dataset necesarios: 'marketing.csv', 'ventas.csv', 'clientes.csv'
import os
os.listdir("/content/drive/MyDrive/datasets")
# Defino las rutas de cada csv para optimizar su acceso
path_ventas    = "/content/drive/MyDrive/datasets/ventas.csv"
path_clientes  = "/content/drive/MyDrive/datasets/clientes.csv"
path_marketing = "/content/drive/MyDrive/datasets/marketing.csv"
# Cargamos los CSV como DataFrames.
df_ventas         = pd.read_csv(path_ventas)
df_clientes       = pd.read_csv(path_clientes)
df_marketing      = pd.read_csv(path_marketing)
# Chequeamos sus dimensiones para comprobar que se cargaron correctamente.
print("Ventas Data Frame Size: ", df_ventas.shape)
print("Clientes Data Frame Size: ", df_clientes.shape)
print("Marketing Data Frame Size: ", df_marketing.shape)
# Mostramos las primeras filas de cada dataset para corroborar estructura de columnas.
# Aplicamos formato de color
display(df_ventas.head(5).style.background_gradient(cmap="Blues"))
display(df_clientes.head(5).style.background_gradient(cmap="Oranges"))
display(df_marketing.head(5).style.background_gradient(cmap="Purples"))

Mounted at /content/drive
Ventas Data Frame Size:  (3035, 6)
Clientes Data Frame Size:  (567, 5)
Marketing Data Frame Size:  (90, 6)


,id_venta,producto,precio,cantidad,fecha_venta,categoria
0,792,Cuadro decorativo,$69.94,5.000000,02/01/2024,Decoración
1,811,Lámpara de mesa,$105.10,5.000000,02/01/2024,Decoración
2,1156,Secadora,$97.96,3.000000,02/01/2024,Electrodomésticos
3,1372,Heladera,$114.35,8.000000,02/01/2024,Electrodomésticos
4,1546,Secadora,$106.21,4.000000,02/01/2024,Electrodomésticos


,id_cliente,nombre,edad,ciudad,ingresos
0,1,Aloysia Screase,44,Mar del Plata,42294.680000
1,2,Kristina Scaplehorn,25,Posadas,24735.040000
2,3,Filip Castagne,50,Resistencia,35744.850000
3,4,Liuka Luard,39,Bahía Blanca,27647.960000
4,5,Dore Cockshtt,28,Rosario,28245.650000


,id_campanha,producto,canal,costo,fecha_inicio,fecha_fin
0,74,Adorno de pared,TV,4.810000,20/03/2024,03/05/2024
1,12,Tablet,RRSS,3.400000,26/03/2024,13/05/2024
2,32,Lámpara de mesa,Email,5.540000,28/03/2024,20/04/2024
3,21,Smartphone,RRSS,6.370000,29/03/2024,16/05/2024
4,58,Alfombra,Email,4.250000,31/03/2024,05/05/2024


## 4- **Introducción a Pandas:**  realizar un análisis exploratorio inicial de los DataFrames.


In [ ]:
def eda(df, nombre):
    # Encabezado con color y emoji
    display(HTML(f"""
    <div style="background-color:#2E86C1; color:white; padding:10px; border-radius:8px; font-size:20px;">
        📊 <b>Análisis exploratorio: {nombre}</b>
    </div>
    """))

    # Info básica
    display(HTML(f"""
    <ul style="font-size:14px; line-height:1.6;">
        <li><b>Shape:</b> {df.shape}</li>
        <li><b>Cantidad de columnas:</b> {len(df.columns)}</li>
        <li><b>Columnas:</b> {list(df.columns)}</li>
    </ul>
    """))

    # Tipos de datos
    display(HTML("<h4 style='color:#8E44AD;'>🧠 Tipos de datos</h4>"))
    display(df.dtypes.to_frame("Tipo de dato").T.style.set_table_styles(
        [{'selector':'th', 'props':[('background-color','#D7BDE2')]}]
    ))

    # Nulos
    display(HTML("<h4 style='color:#E74C3C;'>🚫 Valores nulos por columna</h4>"))
    display(df.isna().sum().to_frame("Nulos").T.style.background_gradient(cmap="Reds"))

    # Primeras filas
    display(HTML("<h4 style='color:#1ABC9C;'>👀 Primeras 5 filas</h4>"))
    display(df.head(5).style.background_gradient(cmap="Blues").set_caption(f"{nombre} - Primeras filas"))

    # Describe
    display(HTML("<h4 style='color:#F39C12;'>📈 Estadísticas descriptivas (numéricas)</h4>"))
    display(df.describe(include='number').style.background_gradient(cmap="Reds").set_caption(f"{nombre} - Describe"))

    # Línea de cierre
    display(HTML("<hr style='border:1px solid #ccc; margin:20px 0;'>"))



In [ ]:
eda(df_ventas, "VENTAS DATA FRAME")
eda(df_clientes, "CLIENTES DATA FRAME")
eda(df_marketing, "MARKETING DATA FRAME")


,id_venta,producto,precio,cantidad,fecha_venta,categoria
Tipo de dato,int64,object,object,float64,object,object


,id_venta,producto,precio,cantidad,fecha_venta,categoria
Nulos,0,0,2,2,0,0


,id_venta,producto,precio,cantidad,fecha_venta,categoria
0,792,Cuadro decorativo,$69.94,5.000000,02/01/2024,Decoración
1,811,Lámpara de mesa,$105.10,5.000000,02/01/2024,Decoración
2,1156,Secadora,$97.96,3.000000,02/01/2024,Electrodomésticos
3,1372,Heladera,$114.35,8.000000,02/01/2024,Electrodomésticos
4,1546,Secadora,$106.21,4.000000,02/01/2024,Electrodomésticos


,id_venta,cantidad
count,3035.000000,3033.000000
mean,1499.851400,6.496538
std,866.465379,3.457250
min,1.000000,1.000000
25%,748.500000,3.000000
50%,1502.000000,7.000000
75%,2249.500000,9.000000
max,3000.000000,12.000000


,id_cliente,nombre,edad,ciudad,ingresos
Tipo de dato,int64,object,int64,object,float64


...

,id_cliente,edad,ingresos
count,567.000000,567.000000,567.000000
mean,284.000000,37.940035,34668.739012
std,163.823075,10.202885,12974.531446
min,1.000000,20.000000,170.290000
25%,142.500000,30.000000,26015.240000
50%,284.000000,37.000000,35066.830000
75%,425.500000,43.000000,42457.100000
max,567.000000,81.000000,88053.010000


,id_campanha,producto,canal,costo,fecha_inicio,fecha_fin
Tipo de dato,int64,object,object,float64,object,object


,id_campanha,producto,canal,costo,fecha_inicio,fecha_fin
Nulos,0,0,0,0,0,0


,id_campanha,producto,canal,costo,fecha_inicio,fecha_fin
0,74,Adorno de pared,TV,4.810000,20/03/2024,03/05/2024
1,12,Tablet,RRSS,3.400000,26/03/2024,13/05/2024
2,32,Lámpara de mesa,Email,5.540000,28/03/2024,20/04/2024
3,21,Smartphone,RRSS,6.370000,29/03/2024,16/05/2024
4,58,Alfombra,Email,4.250000,31/03/2024,05/05/2024


,id_campanha,costo
count,90.000000,90.000000
mean,45.500000,4.928667
std,26.124701,0.947750
min,1.000000,2.950000
25%,23.250000,4.372500
50%,45.500000,4.900000
75%,67.750000,5.562500
max,90.000000,7.390000


---
## 5- **Calidad de Datos:** Identificar valores nulos y duplicados en los conjuntos de datos. Documentar el estado inicial de los datos.

In [ ]:
# Sugerencia: usar .isna().sum() y .duplicated().sum().
# Documentar las observaciones con print() o comentarios.

def calidad(df, nombre, clave=None):
    """
    Función que analiza la calidad del DataFrame:
      - Muestra cantidad de nulos por columna.
      - Cuenta filas duplicadas completas.
      - Si se indica una clave, muestra los valores duplicados más frecuentes.
    Parámetros:
      df: DataFrame de pandas que se analizará.
      nombre: Identificación del DataFrame (ejemplo: 'VENTAS').
      clave: (opcional) nombre de la columna para buscar duplicados específicos.
    """

    # -------------------------------------------------
    # Mostrar título descriptivo con el nombre del DF
    # -------------------------------------------------
    print(f"### {nombre}")

    # -------------------------------------------------
    # Mostrar cantidad de valores nulos por columna
    # -------------------------------------------------
    # Crea una columna llamada "nulos" con la suma de TODOS los elementos
    # nulos de cada columna
    display(df.isna().sum().to_frame("nulos"))

    # -------------------------------------------------
    # Contar filas duplicadas completas
    # -------------------------------------------------
    #Función que reconoce TODAS las filas duplicadas y las suma.
    #Contendrá un numero que representa la cantidad de filas repetidas
    dup_rows = df.duplicated(keep=False).sum()
    print("Filas duplicadas (exactas):", dup_rows)

    # -------------------------------------------------
    # Si se especificó una columna clave válida, analizar duplicados por esa columna
    # -------------------------------------------------
    # Si uno le pasa un valor de clave en la llamada a la función Y esa clave
    # además esta en el DATAFRAME df.columns, entonces cuenta los elementos duplicados
    if clave and clave in df.columns:
        dup_key = df[clave].duplicated(keep=False).sum()
        print(f"Duplicados por clave '{clave}':", dup_key)
        if dup_key > 0:

            # Filtrar filas donde esa clave esté duplicada
            # df[clave].duplicated(keep=False) devuelve True donde el valor se repite
            duplicados_ordenados = (
                df[df[clave].duplicated(keep=False)][clave]
                .value_counts()                # Cuenta cuántas veces aparece cada valor
                .sort_values(ascending=False)   # Ordena de mayor a menor (más duplicados arriba)
            )

            print("\n🔁 Top valores duplicados más frecuentes:")
            # Mostrar solo los primeros 10 (los más repetidos)
            display(duplicados_ordenados.head(10))
        else:
            print(f"No se encontraron duplicados en la clave '{clave}'.")
    else:
        if clave: #si entra aca es que se le dio una clave pero no existe en df
            print(f"La clave '{clave}' no existe en el DataFrame.")
        else: #si entra aca es porque lo anterior es F, es decir, no se le dio clave
            print("No se indicó una clave para analizar duplicados por columna.")
#fin de def calidad

---
# ETAPA 2
---

## 1- **Limpieza de Datos:** Limpiar el conjunto de datos eliminando duplicados y caracteres no deseados. Documentar el proceso y los resultados.

In [ ]:
# TODO: Limpiar el conjunto de datos eliminando duplicados y caracteres no deseados.
# Sugerencia: aplicar .drop_duplicates(), .str.strip(), .str.replace() según el caso.
# Documentar el proceso y los resultados.

## 2- **Transformación de Datos:** Aplicar filtros y transformaciones para crear una tabla de ventas que muestre solo los productos con alto rendimiento.

In [ ]:
# TODO: Aplicar filtros y transformaciones para crear una tabla de ventas
# que muestre solo los productos con alto rendimiento. calcular el percentil 80
# y filtrar los productos que superen ese umbral en ventas.
#quantile(0.8)
# Sugerencia: usar .query() o condiciones con operadores lógicos.

---
## 3- **Agregación:** Resumir las ventas por categoría de producto y analizar los ingresos generados.

In [ ]:
# TODO: Resumir las ventas por categoría de producto y analizar los ingresos generados.
# Sugerencia: usar .groupby() y .agg() para generar métricas como suma y promedio.

---
## 4- **Integración de Datos: (OPCIONAL)** Combinar los sets de datos de ventas y marketing para obtener una visión más amplia de las tendencias.